In [6]:
%cd recipe1M_layers

/Users/shreemit/Developer/RecipeGenerator/RecipeGeneratorNLP/Recipe/recipe1M_layers


In [9]:
!ls

layer1.json layer2.json


In [62]:
import re

In [12]:
import json
with open('layer1.json') as json_file:
    data = json.load(json_file)

In [90]:
recipes = data
recipes[0]['ingredients']

[{'text': '6 ounces penne'},
 {'text': '2 cups Beechers Flagship Cheese Sauce (recipe follows)'},
 {'text': '1 ounce Cheddar, grated (1/4 cup)'},
 {'text': '1 ounce Gruyere cheese, grated (1/4 cup)'},
 {'text': '1/4 to 1/2 teaspoon chipotle chili powder (see Note)'},
 {'text': '1/4 cup (1/2 stick) unsalted butter'},
 {'text': '1/3 cup all-purpose flour'},
 {'text': '3 cups milk'},
 {'text': '14 ounces semihard cheese (page 23), grated (about 3 1/2 cups)'},
 {'text': '2 ounces semisoft cheese (page 23), grated (1/2 cup)'},
 {'text': '1/2 teaspoon kosher salt'},
 {'text': '1/4 to 1/2 teaspoon chipotle chili powder'},
 {'text': '1/8 teaspoon garlic powder'},
 {'text': '(makes about 4 cups)'}]

In [91]:
def get_ingredients(recipe):    
    ingredients = recipe['ingredients']
    measurements = ['oz', 'ounces','ounce', 'cups', 'cup', 'teaspoon', '/', 'to', 'lbs','lb', 'tsp', 'jar']
    item_list = []
    for item in ingredients:
        item = item['text']
    #     item = item.split('(')[0].strip() # remove items in brackets
        item = re.sub(r'\s*\([^)]*\)', '', item)
        item = ''.join([char for char in item if not char.isdigit()]) # remove numbers
        for measurement in measurements:
            item = item.replace(measurement, '').strip() # remove measurements
        item.strip()
        if item:
            item_list.append(item)


    ingredient_list = '|'.join(item_list)
    return ingredient_list

In [92]:
in_recipe = []

for recipe in recipes:
    ingredients = get_ingredients(recipe)
    instr = "test steps"
    qna = { "ingred": ingredients,
            "instructions": instr }
    in_recipe.append(qna)

In [93]:
len(in_recipe)

1029720

In [94]:
in_recipe[0]

{'ingred': 'penne|Beechers Flagship Cheese Sauce|Cheddar, grated|Gruyere cheese, grated|chipotle chili powder|unsalted butter|all-purpose flour|milk|semihard cheese, grated|semisoft cheese, grated|kosher salt|chipotle chili powder|garlic powder',
 'instructions': 'test steps'}